<a href="https://colab.research.google.com/github/adam-bozman/rapidfuzz/blob/main/RapidFuzz_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My own reference
### This was my first attempt at working with a loop, trying to match better accuracy - it didn't work.

In [ ]:
import pandas as pd

In [ ]:
# Import CSV (or XLSX)

# Selecting only the first column

df = pd.read_csv(r"/content/dealer_check.csv", usecols=['dealer_naic'], encoding='mac_roman')
df.rename(columns ={'dealer_naic':'CompanyName'}, inplace = True) 
df = df.iloc[:501, ]

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
from unicodedata import normalize
import re
import nltk
from nltk.corpus import stopwords
from difflib import SequenceMatcher
from sklearn.cluster import AffinityPropagation

In [ ]:
df['CompanyName']=df['CompanyName'].str.lower()

In [ ]:
def remove_whitespace(text):
    return  " ".join(text.split())
df['CompanyName']=df['CompanyName'].apply(remove_whitespace)

In [ ]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

df['CompanyName']=df['CompanyName'].apply(lambda X: word_tokenize(X))

In [ ]:
!pip install spellchecker -q

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
def spell_check(text):
    
    result = []
    spell = SpellChecker()
    for word in text:
        correct_word = spell.correction(word)
        result.append(correct_word)
    
    return result

df['CompanyName'] = df['CompanyName'].apply(spell_check)

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

en_stopwords = stopwords.words('english')

def remove_stopwords(text):
    result = []
    for token in text:
        if token not in en_stopwords:
            result.append(token)
            
    return result
  
df['CompanyName'] = df['CompanyName'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import RegexpTokenizer

def remove_punct(text):
    
    tokenizer = RegexpTokenizer(r"\w+")
    lst=tokenizer.tokenize(' '.join(text))
    return lst

df['CompanyName'] = df['CompanyName'].apply(remove_punct)

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag

def lemmatization(text):
    
    result=[]
    wordnet = WordNetLemmatizer()
    for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))
    
    return result

df['CompanyName']=df['CompanyName'].apply(lemmatization)

In [ ]:
from nltk.stem import PorterStemmer

def stemming(text):
    porter = PorterStemmer()
    
    result=[]
    for word in text:
        result.append(porter.stem(word))
    return result

df['CompanyName']=df['CompanyName'].apply(stemming)

In [ ]:
import re
def remove_tag(text):
    
    text=' '.join(text)
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)
df['CompanyName'] = df['CompanyName'].apply(remove_tag)

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

df['CompanyName'] = df['CompanyName'].apply(remove_urls)

In [ ]:
from collections import Counter
all_names = df['CompanyName'].unique()
names_freq = Counter()
for name in all_names:
    names_freq.update(str(name).split(" "))
key_words = [word for (word,_) in names_freq.most_common(30)]
print(key_words)

['abn', 'amro', '1st', 'bank', 'inc', 'secur', 'corp', 'bond', 'acac', 'co', 'trade', 'nation', '1stbank', 'g', 'chicago', 'nat', 'usa', 'ambro', 'trust', 'gwla', '6', 'in', 'sec', 'abg', 'a', 'aam', 'inact', 'capit', 'edward', 'aasi']


In [ ]:
all_main_name = pd.DataFrame(columns=['names','alias','score'])
all_names.sort()
all_main_name['names'] = all_names
all_main_name['sort_gp'] = all_main_name['names'].apply(lambda x: x[-1:])

In [ ]:
!pip install rapidfuzz -q

In [ ]:
from rapidfuzz import fuzz

all_sort_gp = all_main_name['names'].unique()

def no_key_word(name):
    """check if the name contain the keywords in travel company"""
    output = True
    for key in key_words:
        if key in name:
            output = False
    return output

for sortgp in all_sort_gp:
    this_gp = all_main_name.groupby(all_main_name['names'], all_main_name['score'])
    gp_start = this_gp.index.min()
    gp_end = this_gp.index.max()
    for i in range(gp_start,gp_end+1):
    
        # if self has not got alias, asign to be alias of itself
        if pd.isna(all_main_name['alias'].iloc[i]):
            all_main_name['alias'].iloc[i] = all_main_name['names'].iloc[i]
            all_main_name['score'].iloc[i] = 100
        
        # if the following has not got alias and fuzzy match, asign to be alias of this one
        for j in range(i+1,gp_end+1):
            if pd.isna(all_main_name['alias'].iloc[j]):
                fuzz_socre = fuzz.token_sort_ratio(all_main_name['names'].iloc[i],all_main_name['names'].iloc[j])
                if not no_key_word(all_main_name['names'].iloc[j]):
                    fuzz_socre -= 10
                if (fuzz_socre > 85):
                    all_main_name['alias'].iloc[j] = all_main_name['alias'].iloc[i]
                    all_main_name['score'].iloc[j] = fuzz_socre
                    
        if i % (len(all_names)//10) == 0:
            print("progress: %.2f" % (100*i/len(all_names)) + "%")

TypeError: ignored

In [ ]:
all_main_name.head(50)

,names,alias,score
0,,,100
1,00120 euroclear 120,00120 euroclear 120,100
2,002 fpa,002 fpa,100
3,002177 9126 rein ireland,002177 9126 rein ireland,100
4,003 five year mva annuiti,003 five year mva annuiti,100
5,004 money market annuiti,004 money market annuiti,100
6,009 univers life,009 univers life,100
7,030 ameru full safe aka 4,030 ameru full safe aka 4,100
8,031 ameru full safe aka 8,031 ameru full safe aka 8,100
9,037 ameru full safe aka 21,037 ameru full safe aka 21,100
